In [2]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os

from six.moves import xrange 
import tensorflow as tf
import numpy as np
import time

# Process images of this size. Note that this differs from the original CIFAR
# image size of 32 x 32. If one alters this number, then the entire model
# architecture will change and any model would need to be retrained.
IMAGE_SIZE = 24

# Global constants describing the CIFAR-10 data set.
NUM_CLASSES = 10
NUM_EXAMPLES_PER_EPOCH_FOR_TRAIN = 50000
NUM_EXAMPLES_PER_EPOCH_FOR_EVAL = 10000


def read_cifar10(filename_queue):
    """Reads and parses examples from CIFAR10 data files.

    Recommendation: if you want N-way read parallelism, call this function
    N times.  This will give you N independent Readers reading different
    files & positions within those files, which will give better mixing of
    examples.

    Args:
        filename_queue: A queue of strings with the filenames to read from.

    Returns:
        An object representing a single example, with the following fields:
            height: number of rows in the result (32)
            width: number of columns in the result (32)
            depth: number of color channels in the result (3)
            key: a scalar string Tensor describing the filename & record number
                for this example.
            label: an int32 Tensor with the label in the range 0..9.
            uint8image: a [height, width, depth] uint8 Tensor with the image data
    """

    class CIFAR10Record(object):
        pass
    result = CIFAR10Record()

    label_bytes = 1
    result.height = 32
    result.width = 32
    result.depth = 3
    image_bytes = result.height * result.width * result.depth
    # Every record consists of a label followed by the image, with a
    # fixed number of bytes for each.
    record_bytes = label_bytes + image_bytes

    # Read a record, getting filenames from the filename_queue.  No
    # header or footer in the CIFAR-10 format, so we leave header_bytes
    # and footer_bytes at their default of 0.
    reader = tf.FixedLengthRecordReader(record_bytes=record_bytes)
    result.key, value = reader.read(filename_queue)

    # Convert from a string to a vector of uint8 that is record_bytes long.
    record_bytes = tf.decode_raw(value, tf.uint8)

    # The first bytes represent the label, which we convert from uint8->int32.
    result.label = tf.cast(
            tf.strided_slice(record_bytes, [0], [label_bytes]), tf.int32)

    # The remaining bytes after the label represent the image, which we reshape
    # from [depth * height * width] to [depth, height, width].
    depth_major = tf.reshape(
            tf.strided_slice(record_bytes, [label_bytes],
                                             [label_bytes + image_bytes]),
            [result.depth, result.height, result.width])
    # Convert from [depth, height, width] to [height, width, depth].
    result.uint8image = tf.transpose(depth_major, [1, 2, 0])

    return result


def _generate_image_and_label_batch(image, label, min_queue_examples,
                                    batch_size, shuffle):
    """Construct a queued batch of images and labels.

    Args:
        image: 3-D Tensor of [height, width, 3] of type.float32.
        label: 1-D Tensor of type.int32
        min_queue_examples: int32, minimum number of samples to retain
            in the queue that provides of batches of examples.
        batch_size: Number of images per batch.
        shuffle: boolean indicating whether to use a shuffling queue.

    Returns:
        images: Images. 4D tensor of [batch_size, height, width, 3] size.
        labels: Labels. 1D tensor of [batch_size] size.
    """
    # Create a queue that shuffles the examples, and then
    # read 'batch_size' images + labels from the example queue.
    num_preprocess_threads = 8
    if shuffle:
        images, label_batch = tf.train.shuffle_batch(
                [image, label],
                batch_size=batch_size,
                num_threads=num_preprocess_threads,
                capacity=min_queue_examples + 3 * batch_size,
                min_after_dequeue=min_queue_examples)
    else:
        images, label_batch = tf.train.batch(
                [image, label],
                batch_size=batch_size,
                num_threads=num_preprocess_threads,
                capacity=min_queue_examples + 3 * batch_size)

    return images, tf.reshape(label_batch, [batch_size])

def get_inputs(data_dir, batch_size, is_test=False):
    """Construct distorted input for CIFAR training using the Reader ops.

    Args:
        data_dir: Path to the CIFAR-10 data directory.
        batch_size: Number of images per batch.

    Returns:
        images: Images. 4D tensor of [batch_size, IMAGE_SIZE, IMAGE_SIZE, 3] size.
        labels: Labels. 1D tensor of [batch_size] size.
    """
    if not is_test:
        filenames = [os.path.join(data_dir, 'data_batch_%d.bin' % i)
                                 for i in xrange(1, 6)]
        num_examples_per_epoch = NUM_EXAMPLES_PER_EPOCH_FOR_TRAIN
    else:
        filenames = [os.path.join(data_dir, 'test_batch.bin')]
        num_examples_per_epoch = NUM_EXAMPLES_PER_EPOCH_FOR_EVAL

    for f in filenames:
        if not tf.gfile.Exists(f):
            raise ValueError('Failed to find file: ' + f)

    # Create a queue that produces the filenames to read.
    filename_queue = tf.train.string_input_producer(filenames)

    # Read examples from files in the filename queue.
    read_input = read_cifar10(filename_queue)
    reshaped_image = tf.cast(read_input.uint8image, tf.float32)

    height = IMAGE_SIZE
    width = IMAGE_SIZE

    if not is_test:
        ##### NO AUGMENTATION
        # distorted_image = tf.image.resize_image_with_crop_or_pad(reshaped_image, height, width)

        ##### AUGMENTATION
        # randomly crop 24x24x3 out of the 32x32x3 input
        distorted_image = tf.random_crop(reshaped_image, [height, width, 3])
        # flip the image
        distorted_image = tf.image.random_flip_left_right(distorted_image)
        # random brightness
        distorted_image = tf.image.random_brightness(distorted_image,max_delta=63)
        # random contrast
        distorted_image = tf.image.random_contrast(distorted_image,lower=0.2, upper=1.8)
    else:
        # crops just the center
        distorted_image = tf.image.resize_image_with_crop_or_pad(reshaped_image, height, width)

    # normalizes the inputs (originally 0-255)
    float_image = tf.image.per_image_standardization(distorted_image)

    # Set the shapes of tensors.
    float_image.set_shape([height, width, 3])
    read_input.label.set_shape([1])

    if not is_test:
        min_fraction_of_examples_in_queue = 0.4
        min_queue_examples = int(NUM_EXAMPLES_PER_EPOCH_FOR_TRAIN *
                                                         min_fraction_of_examples_in_queue)
        print ('Filling training queue with %d CIFAR images before starting to train. '
                     'This will take a few minutes.' % min_queue_examples)
    else:
        min_fraction_of_examples_in_queue = 0.4
        min_queue_examples = int(NUM_EXAMPLES_PER_EPOCH_FOR_EVAL *
                                                         min_fraction_of_examples_in_queue)
        print ('Filling testing queue with %d CIFAR images.' % min_queue_examples)

    # Generate a batch of images and labels by building up a queue of examples.
    return _generate_image_and_label_batch(float_image,read_input.label,min_queue_examples,batch_size,shuffle=True)

def train_inputs(batch_size):
    data_dir = 'cifar10_data/cifar-10-batches-bin/'
    images, labels = get_inputs(data_dir=data_dir,batch_size=batch_size)
    return images, labels

def test_inputs(batch_size):
    data_dir = 'cifar10_data/cifar-10-batches-bin/'
    images, labels = get_inputs(data_dir=data_dir,batch_size=batch_size, is_test=True)
    return images, labels

import numpy as np
import os

#from model import cifar10_conv

batch_size = 64

#### TRAIN
with tf.device('/cpu:0'):
    images, labels = train_inputs(batch_size)
    images_test, labels_test = test_inputs(batch_size)

with tf.variable_scope('placeholder'):
    X = tf.placeholder(tf.float32, [None, IMAGE_SIZE, IMAGE_SIZE, 3])
    y = tf.placeholder(name='label',dtype=tf.float32,shape=[batch_size,10])
    keep_prob = tf.placeholder(tf.float32 ,shape=())

with tf.variable_scope('model'):
    output = cifar10_conv(X, keep_prob=keep_prob)

with tf.variable_scope('loss'):
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=output,labels=y))

with tf.variable_scope('accuracy'):
    correct_prediction = tf.equal(tf.argmax(output,1), tf.argmax(y,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

tvar = tf.trainable_variables()
cifar10_var = [var for var in tvar if 'cifar10_conv' in var.name]

with tf.name_scope('train'):
    train_step = tf.train.AdamOptimizer(learning_rate=1e-3).minimize(loss, var_list=cifar10_var)

saver = tf.train.Saver(tvar)

sess = tf.InteractiveSession()

init = tf.global_variables_initializer()
sess.run(init)


if not os.path.exists('current_model/'):
    os.makedirs('current_model/')

# saver.restore(sess,tf.train.latest_checkpoint('current_model/'))

coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(coord=coord)

keep_probability = 0.7

tf.train.start_queue_runners()
loss_print = 0
accuracy_print = 0
t = time.time()
for i in range(0,3):

    X_batch, labels_batch = sess.run([images, labels])

    y_batch = np.zeros((batch_size,NUM_CLASSES))
    y_batch[np.arange(batch_size),labels_batch] = 1

    _, loss_print, accuracy_print = sess.run([train_step, loss, accuracy], feed_dict={X: X_batch, y: y_batch, keep_prob:keep_probability})

    if i % 20 == 0:
        print('time: %f iteration:%d loss:%f accuracy:%f' % (float(time.time()-t), i, loss_print, accuracy_print))
        t = time.time()

    if i % 500 == 0:

        test_accuracy = 0.0
        accuracy_count = 0

        for j in xrange(50):
            X_batch, labels_batch = sess.run([images_test,labels_test])
            y_batch = np.zeros((batch_size,NUM_CLASSES))
            y_batch[np.arange(batch_size),labels_batch] = 1

            accuracy_print = sess.run([accuracy], feed_dict={X: X_batch, y: y_batch, keep_prob:1.0})

            test_accuracy += accuracy_print[0]
            accuracy_count += 1
        test_accuracy = test_accuracy/accuracy_count
        print('TEST:%f' % test_accuracy)
    if i % 1 == 0:
        with open('HW2_Output.txt','a') as text_file:
            text_file.write("jjTEST: %f\n" %(test_accuracy))

saver.save(sess, 'current_model/model',global_step=2)

'''
### JUST TEST
tf.train.start_queue_runners()
t = time.time()
test_accuracy = 0.0
accuracy_count = 0
for i in range(10000):
    X_batch, labels_batch = sess.run([images_test, labels_test])
    
    y_batch = np.zeros((batch_size,NUM_CLASSES))
    y_batch[np.arange(batch_size),labels_batch] = 1

    accuracy_print = sess.run([accuracy], feed_dict={X: X_batch, y: y_batch, keep_prob:1.0})

    test_accuracy += accuracy_print[0]
    accuracy_count += 1
    if i % 10 == 0:
        print('time: %f accuracy:%f' % (float(time.time()-t),test_accuracy/accuracy_count))
        t = time.time()
        
with open('HW2_Output.txt','a') as text_file:
    text_file.write("The test accuracy of this model is: %f" %(test_accuracy/accuracy_count))
'''

Filling training queue with 20000 CIFAR images before starting to train. This will take a few minutes.
Filling testing queue with 4000 CIFAR images.


KeyboardInterrupt: 

In [1]:
#### modified version ############
import tensorflow as tf


def conv(x, w, b, stride, name):
    with tf.variable_scope('conv'):
        return tf.nn.conv2d(x,
                            filter=w,
                            strides=[1, stride, stride, 1],
                            padding='SAME',
                            name=name) + b


######## after 30k iterations (batch_size=64)
# with data augmentation (flip, brightness, contrast) ~81.0%
# without data augmentation 69.6%
def cifar10_conv(X, keep_prob, reuse=False):
    with tf.variable_scope('cifar10_conv'):
        if reuse:
            tf.get_variable_scope().reuse_variables()

        batch_size = tf.shape(X)[0]
        K1 = 32
        K2 = 32
        K3 = 32
        K4 = 48
        K5 = 48
        K6 = 80
        K7 = 80
        K8 = 80
        K9 = 80
        K10 = 80
        K11 = 128
        K12 = 128
        K13 = 128
        K14 = 128
        K15 = 128
        T = 73728
        K16 = 500
        
        W1 = tf.get_variable('D_W1', [3, 3, 3, K1], initializer=tf.contrib.layers.xavier_initializer())
        B1 = tf.get_variable('D_B1', [K1], initializer=tf.constant_initializer())
        conv1 = conv(X, W1, B1, stride=1, name='conv1')
        bn1 = tf.nn.relu(tf.contrib.layers.batch_norm(conv1))

        W2 = tf.get_variable('D_W2', [3, 3, K1, K2], initializer=tf.contrib.layers.xavier_initializer())
        B2 = tf.get_variable('D_B2', [K2], initializer=tf.constant_initializer())
        conv2 = conv(bn1, W2, B2, stride=1, name='conv2')
        bn2 = tf.nn.relu(tf.contrib.layers.batch_norm(conv2))

        W3 = tf.get_variable('D_W3', [3, 3, K2, K3], initializer=tf.contrib.layers.xavier_initializer())
        B3 = tf.get_variable('D_B3', [K3], initializer=tf.constant_initializer())
        conv3 = conv(bn2, W3, B3, stride=1, name='conv3')
        bn3 = tf.nn.relu(tf.contrib.layers.batch_norm(conv3))
        
        W4 = tf.get_variable('D_W4', [3, 3, K3, K4], initializer=tf.contrib.layers.xavier_initializer())
        B4 = tf.get_variable('D_B4', [K4], initializer=tf.constant_initializer())
        conv4 = conv(bn3, W4, B4, stride=1, name='conv4')
        bn4 = tf.nn.relu(tf.contrib.layers.batch_norm(conv4))
        
        W5 = tf.get_variable('D_W5', [3, 3, K4, K5], initializer=tf.contrib.layers.xavier_initializer())
        B5 = tf.get_variable('D_B5', [K5], initializer=tf.constant_initializer())
        conv5 = conv(bn4, W5, B5, stride=1, name='conv5')
        bn5 = tf.nn.relu(tf.contrib.layers.batch_norm(conv5))
        pooled5 = tf.nn.max_pool(bn5, ksize=[1, 2, 2, 1], strides=[1, 1, 1, 1], padding='SAME')
        d5 = tf.nn.dropout(pooled5, keep_prob)
        
        W6 = tf.get_variable('D_W6', [3, 3, K5, K6], initializer=tf.contrib.layers.xavier_initializer())
        B6 = tf.get_variable('D_B6', [K6], initializer=tf.constant_initializer())
        conv6 = conv(d5, W6, B6, stride=1, name='conv6')
        bn6 = tf.nn.relu(tf.contrib.layers.batch_norm(conv6))
        
        W7 = tf.get_variable('D_W7', [3, 3, K6, K7], initializer=tf.contrib.layers.xavier_initializer())
        B7 = tf.get_variable('D_B7', [K7], initializer=tf.constant_initializer())
        conv7 = conv(bn6, W7, B7, stride=1, name='conv7')
        bn7 = tf.nn.relu(tf.contrib.layers.batch_norm(conv7))
        
        W8 = tf.get_variable('D_W8', [3, 3, K7, K8], initializer=tf.contrib.layers.xavier_initializer())
        B8 = tf.get_variable('D_B8', [K8], initializer=tf.constant_initializer())
        conv8 = conv(bn7, W8, B8, stride=1, name='conv8')
        bn8 = tf.nn.relu(tf.contrib.layers.batch_norm(conv8))
        
        W9 = tf.get_variable('D_W9', [3, 3, K8, K9], initializer=tf.contrib.layers.xavier_initializer())
        B9 = tf.get_variable('D_B9', [K9], initializer=tf.constant_initializer())
        conv9 = conv(bn8, W9, B9, stride=1, name='conv9')
        bn9 = tf.nn.relu(tf.contrib.layers.batch_norm(conv9))
        
        W10 = tf.get_variable('D_W10', [3, 3, K9, K10], initializer=tf.contrib.layers.xavier_initializer())
        B10 = tf.get_variable('D_B10', [K10], initializer=tf.constant_initializer())
        conv10 = conv(bn9, W10, B10, stride=1, name='conv10')
        bn10 = tf.nn.relu(tf.contrib.layers.batch_norm(conv10))
        pooled10 = tf.nn.max_pool(bn10, ksize=[1, 2, 2, 1], strides=[1, 1, 1, 1], padding='SAME')
        d10 = tf.nn.dropout(pooled10, keep_prob)
        
        
        W11 = tf.get_variable('D_W11', [3, 3, K10, K11], initializer=tf.contrib.layers.xavier_initializer())
        B11 = tf.get_variable('D_B11', [K11], initializer=tf.constant_initializer())
        conv11 = conv(d10, W11, B11, stride=1, name='conv11')
        bn11 = tf.nn.relu(tf.contrib.layers.batch_norm(conv11))
        
        W12 = tf.get_variable('D_W12', [3, 3, K11, K12], initializer=tf.contrib.layers.xavier_initializer())
        B12 = tf.get_variable('D_B12', [K12], initializer=tf.constant_initializer())
        conv12 = conv(bn11, W12, B12, stride=1, name='conv12')
        bn12 = tf.nn.relu(tf.contrib.layers.batch_norm(conv12))
        
        W13 = tf.get_variable('D_W13', [3, 3, K12, K13], initializer=tf.contrib.layers.xavier_initializer())
        B13 = tf.get_variable('D_B13', [K13], initializer=tf.constant_initializer())
        conv13 = conv(bn12, W13, B13, stride=1, name='conv13')
        bn13 = tf.nn.relu(tf.contrib.layers.batch_norm(conv13))
        
        W14 = tf.get_variable('D_W14', [3, 3, K13, K14], initializer=tf.contrib.layers.xavier_initializer())
        B14 = tf.get_variable('D_B14', [K14], initializer=tf.constant_initializer())
        conv14 = conv(bn13, W14, B14, stride=1, name='conv14')
        bn14 = tf.nn.relu(tf.contrib.layers.batch_norm(conv14))
        
        W15 = tf.get_variable('D_W15', [3, 3, K14, K15], initializer=tf.contrib.layers.xavier_initializer())
        B15 = tf.get_variable('D_B15', [K15], initializer=tf.constant_initializer())
        conv15 = conv(bn14, W15, B15, stride=1, name='conv15')
        bn15 = tf.nn.relu(tf.contrib.layers.batch_norm(conv15))
        pooled15 = tf.nn.max_pool(bn15, ksize=[1, 2, 2, 1], strides=[1, 1, 1, 1], padding='SAME')
        d15 = tf.nn.dropout(pooled15, keep_prob)
        
        
        flat = tf.reshape(d15, [batch_size, T])
    
        W16 = tf.get_variable('D_W16', [T, K16], initializer=tf.contrib.layers.xavier_initializer())
        B16 = tf.get_variable('D_B16', [K16], initializer=tf.constant_initializer())
        M16 = tf.matmul(flat, W16) + B16
        bn16 = tf.nn.relu(tf.contrib.layers.batch_norm(M16))
        d16 = tf.nn.dropout(bn16, keep_prob)
        
        
        W17 = tf.get_variable('D_W17', [K16, 10], initializer=tf.contrib.layers.xavier_initializer())
        B17 = tf.get_variable('D_B17', [10], initializer=tf.constant_initializer())
        M17 = tf.matmul(d16, W17) + B17
        output = tf.nn.softmax(M17)
        
        return output

In [1]:
############ original ############################
import tensorflow as tf


def conv(x, w, b, stride, name):
    with tf.variable_scope('conv'):
        return tf.nn.conv2d(x,
                            filter=w,
                            strides=[1, stride, stride, 1],
                            padding='SAME',
                            name=name) + b


######## after 30k iterations (batch_size=64)
# with data augmentation (flip, brightness, contrast) ~81.0%
# without data augmentation 69.6%
def cifar10_conv(X, keep_prob, reuse=False):
    with tf.variable_scope('cifar10_conv'):
        if reuse:
            tf.get_variable_scope().reuse_variables()

        batch_size = tf.shape(X)[0]
        K = 64
        M = 128
        N = 256

        W1 = tf.get_variable('D_W1', [5, 5, 3, K], initializer=tf.contrib.layers.xavier_initializer())
        B1 = tf.get_variable('D_B1', [K], initializer=tf.constant_initializer())

        W2 = tf.get_variable('D_W2', [5, 5, K, M], initializer=tf.contrib.layers.xavier_initializer())
        B2 = tf.get_variable('D_B2', [M], initializer=tf.constant_initializer())

        W3 = tf.get_variable('D_W3', [5, 5, M, N], initializer=tf.contrib.layers.xavier_initializer())
        B3 = tf.get_variable('D_B3', [N], initializer=tf.constant_initializer())

        W4 = tf.get_variable('D_W4', [N, 10], initializer=tf.contrib.layers.xavier_initializer())
        B4 = tf.get_variable('D_B4', [10], initializer=tf.constant_initializer())

        conv1 = conv(X, W1, B1, stride=2, name='conv1')
        bn1 = tf.nn.relu(tf.contrib.layers.batch_norm(conv1))
        print('bn1')
        print(bn1.shape)

        conv2 = conv(bn1, W2, B2, stride=2, name='conv2')
        bn2 = tf.nn.relu(tf.contrib.layers.batch_norm(conv2))
        print('bn2')
        print(bn2.shape)

        conv3 = conv(bn2, W3, B3, stride=2, name='conv3')
        bn3 = tf.nn.dropout(tf.nn.relu(tf.contrib.layers.batch_norm(conv3)), keep_prob)
        print('bn3')
        print(bn3.shape)

        pooled = tf.nn.avg_pool(bn3, ksize=[1, 3, 3, 1], strides=[1, 1, 1, 1], padding='VALID')
        print('pooled')
        print(pooled.shape)
        

        flat = tf.reshape(pooled, [batch_size, N])
        print('flat')
        print(flat.shape)
        output = tf.matmul(flat, W4) + B4
        print('output')
        print(output.shape)

        # return tf.nn.softmax(output)
        return output

In [4]:
a = 0.98*0.98*0.03
b = 0.04*0.04*0.97
a/(a+b)

0.9488868396785667